# LSTM으로 지문을 읽어서 주제를 분류하기

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from IPython.display import Image

LSTM을 사용하여 각 지문의 주제를 분류하는 예제를 다뤄보겠습니다.  
아래 실습을 위해 입력값을 팬더스 데이터프레임에 저장합니다.

In [3]:
paragraph_dict_list = [
         {'paragraph': 'dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait', 'category': 'food'},
         {'paragraph': 'service can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations', 'category': 'food'},
         {'paragraph': 'portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table', 'category': 'food'},
         {'paragraph': 'we started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition', 'category': 'food'},
         {'paragraph': 'the biscuits and gravy was too salty two people in my group had the gravy and all thought it was too salty my hubby ordered a side of double egg and it was served on two small plates who serves eggs to one person on separate plates we commented on that when it was delivered and even the server laughed and said she doesnt know why the kitchen does that presentation of food is important and they really missed on this one', 'category': 'food'},
         {'paragraph': 'the garlic fries were a great starter (and a happy hour special) the pancakes looked and tasted great and were a fairly generous portion', 'category': 'food'},
         {'paragraph': 'our meal was excellent i had the pasta ai formaggi which was so rich i didnt dare eat it all although i certainly wanted to excellent flavors with a great texture contrast between the soft pasta and the crisp bread crumbs too much sauce for me but a wonderful dish', 'category': 'food'},
         {'paragraph': 'what i enjoy most about palo alto is so many restaurants have dog-friendly seating outside i had bookmarked italico from when they first opened about a 1.5 years ago and was jonesing for some pasta so time to finally knock that bookmark off', 'category': 'food'},
         {'paragraph': 'the drinks came out fairly quickly a good two to three minutes after the orders were taken i expected my iced tea to taste a bit more sweet but this was straight up green tea with ice in it not to complain of course but i was pleasantly surprised', 'category': 'food'},
         {'paragraph': 'despite the not so good burger the service was so slow the restaurant wasnt even half full and they took very long from the moment we got seated to the time we left it was almost 2 hours we thought that it would be quick since we ordered as soon as we sat down my coworkers did seem to enjoy their beef burgers for those who eat beef however i will not be returning it is too expensive and extremely slow service', 'category': 'food'},
    
         {'paragraph': 'the four reigning major champions simona halep caroline wozniacki angelique kerber and defending us open champion sloane stephens could make a case for being the quartet most likely to succeed especially as all but stephens has also enjoyed the no1 ranking within the last 14 months as they prepare for their gruelling new york campaigns they currently hold the top four places in the ranks', 'category': 'sports'},
         {'paragraph': 'the briton was seeded nn7 here last year before a slump in form and confidence took her down to no46 after five first-round losses but there have been signs of a turnaround including a victory over a sub-par serena williams in san jose plus wins against jelena ostapenko and victoria azarenka in montreal. konta pulled out of new haven this week with illness but will hope for good things where she first scored wins in a major before her big breakthroughs to the semis in australia and wimbledon', 'category': 'sports'},
         {'paragraph': 'stephens surged her way back from injury in stunning style to win her first major here last year—and ranked just no83 she has since proved what a big time player she is winning the miami title via four fellow major champions then reaching the final at the french open back on north american hard courts she ran to the final in montreal only just edged out by halep she has also avoided many of the big names in her quarter—except for wild card azarenka as a possible in the third round', 'category': 'sports'},
         {'paragraph': 'when it came to england chances in the world cup it would be fair to say that most fans had never been more pessimistic than they were this year after enduring years of truly dismal performances at major tournaments – culminating in the 2014 event where they failed to win any of their three group games and finished in bottom spot those results led to the resignation of manager roy hodgson', 'category': 'sports'},
         {'paragraph': 'the team that eliminated russia – croatia – also improved enormously during the tournament before it began their odds were 33/1 but they played with real flair and star players like luka modric ivan rakitic and ivan perisic showed their quality on the world stage having displayed their potential by winning all three of their group stage games croatia went on to face difficult tests like the semi-final against england', 'category': 'sports'},
         {'paragraph': 'the perseyside outfit finished in fourth place in the premier league table and without a trophy last term after having reached the champions league final before losing to real madrid', 'category': 'sports'},
         {'paragraph': 'liverpool fc will return to premier league action on saturday lunchtime when they travel to leicester city in the top flight as they look to make it four wins in a row in the league', 'category': 'sports'},
         {'paragraph': 'alisson signed for liverpool fc from as roma this summer and the brazilian goalkeeper has helped the reds to keep three clean sheets in their first three premier league games', 'category': 'sports'},
         {'paragraph': 'but the rankings during that run-in to new york hid some very different undercurrents for murray had struggled with a hip injury since the clay swing and had not played a match since losing his quarter-final at wimbledon and he would pull out of the us open just two days before the tournament began—too late however to promote nederer to the no2 seeding', 'category': 'sports'},
         {'paragraph': 'then came the oh-so-familiar djokovic-nadal no-quarter-given battle for dominance in the third set there were exhilarating rallies with both chasing to the net both retrieving what looked like winning shots nadal more than once pulled off a reverse smash and had his chance to seal the tie-break but it was djokovic serving at 10-9 who dragged one decisive error from nadal for a two-sets lead', 'category': 'sports'}
]
df = pd.DataFrame(paragraph_dict_list)
df = df[['paragraph', 'category']]

주제가 **음식**인 지문을 몇개 출력해보도록 하겠습니다.

In [5]:
df.head()

,paragraph,category
0,dishplace is located in sunnyvale downtown the...,food
1,service can be slower during busy hours but ou...,food
2,portions are huge both french toast and their ...,food
3,we started with apps going the chicken and waf...,food
4,the biscuits and gravy was too salty two peopl...,food


주제가 **스포츠**엔 지문을 몇개 출력해보도록 하겠습니다.

In [6]:
df.tail()

,paragraph,category
15,the perseyside outfit finished in fourth place...,sports
16,liverpool fc will return to premier league act...,sports
17,alisson signed for liverpool fc from as roma t...,sports
18,but the rankings during that run-in to new yor...,sports
19,then came the oh-so-familiar djokovic-nadal no...,sports


# 데이터 전처리
모든 딥러닝 모델들이 수학에 근간하듯이, LSTM 역시 수학을 기반으로한 모델이므로, 입력값을 수치값으로 변경해주어야 합니다. 텍스트인 입력값을 수치로 변경하기 위해, 지문에 사용된 모든 단어들을 모아 중복을 제거하여 단어 리스트를 만듭니다

In [7]:
results = set()
df['paragraph'].str.lower().str.split().apply(results.update)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
Name: paragraph, dtype: object

리스트에 들어간 순서(인덱스)가 곧 단어를 표현하는 숫자가 됩니다. word2idx 딕셔너리를 생성하여 추후 쉽게 단어를 숫자로 전환할 수 있도록 하겠습니다.

In [8]:
idx2word= dict(enumerate(results))
word2idx = {v: k for k, v in idx2word.items()}

예를 들어, 만약 문장이 "one week"일 경우, 이번 실습을 통해 구현될 모델의 입력값은 [15, 4]가 됩니다.  
아래 코드를 실행하여 "bread"의 인덱스를 확인해보겠습니다.

In [9]:
word2idx["bread"]

456

아래 코드를 실행하여 "tournament"의 인덱스를 확인해보겠습니다.

In [10]:
word2idx["tournament"]

451

word2idx를 활용하여 모든 지문을 수치로 전환하도록 하겠습니다.

In [11]:
def encode_paragraph(paragraph):
    words = paragraph.split(" ")
    encoded = []
    for word in words:
        encoded.append([word2idx[word]])
    
    return encoded

df["enc_paragraph"] = df.paragraph.apply(encode_paragraph)

분류 항목(food, sports) 역시 수치로 변경해야합니다. 분류 항목은 one hot encoding으로 변경합니다.

In [12]:
def encode_category(category):
    if category == "food":
        return [1,0]
    else:
        return [0,1]
    
df["enc_category"] = df.category.apply(encode_category)

이번 실습에 사용할 RNN은 Dynamic RNN입니다. Dynamic RNN은 입력값의 다양한 길이를 고려하여 결과값을 출력합니다.  
Dynamic RNN이 각 입력값의 실제 길이(단어수)를 알 수 있도록 각 지문별 단어수를 미리 계산해놓도록 하겠습니다.

In [13]:
def word_cnt(paragraph):
    return len(paragraph.split(" "))

df["seq_length"] = df.paragraph.apply(word_cnt)

텐서플로우의 RNN은 항상 같은 길이의 입력 시퀀스를 받아야하므로, 길이가 작은 입력 시퀀스는 패딩을 추가적으로 넣어서 모든 시퀀스의 길이를 동일하게 설정합니다. 패딩이 RNN 계산에 영향을 끼치지 않도록, Dynamic RNN은 패딩 이전의 입력 시퀀스의 실제길이를 파라미터로 받아 계산에서 패딩을 제외시킵니다.  

이번 실습에서는 최고로 긴 지문의 단어수를 구한 후, 모든 지문에 패딩을 집어넣어 최고로 긴 지문과 동일한 길이를 갖도록 하겠습니다.  
아래의 코드를 실행하여 최고로 긴 지문의 단어수를 구합니다.

In [16]:
max_word_cnt = 0
for row in df['paragraph']:
    if len(row.split(" ")) > max_word_cnt:
        max_word_cnt = len(row.split(" "))

print (max_word_cnt)

91


모든 지문에 패딩을 집어넣어 최고로 긴 지문과 동일한 길이를 갖도록 하겠습니다.

In [18]:
def sequence_padding(enc_paragraph):
    seq_length = len(enc_paragraph)
    for i in range(seq_length,max_word_cnt):
        enc_paragraph.append([-1])
    
    return enc_paragraph

df["enc_paragraph"] = df.enc_paragraph.apply(sequence_padding)

아래의 코드를 실행하여, 패딩이 들어간 문장을 확인해보도록 하겠습니다.

In [19]:
print(df["enc_paragraph"][0])

[[512], [228], [486], [385], [367], [346], [96], [228], [320], [40], [299], [88], [164], [472], [125], [350], [356], [476], [370], [181], [359], [160], [389], [306], [485], [363], [318], [221], [476], [382], [114], [435], [39], [198], [383], [219], [22], [6], [447], [174], [187], [318], [502], [530], [355], [9], [534], [349], [504], [299], [12], [476], [336], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1]]


이제, 수치화된 모든 데이터를 확인해볼 수 있습니다.

In [21]:
df.head(20)

,paragraph,category,enc_paragraph,enc_category,seq_length
0,dishplace is located in sunnyvale downtown the...,food,"[[512], [228], [486], [385], [367], [346], [96...","[1, 0]",53
1,service can be slower during busy hours but ou...,food,"[[459], [125], [350], [250], [181], [174], [38...","[1, 0]",19
2,portions are huge both french toast and their ...,food,"[[327], [522], [69], [352], [152], [388], [363...","[1, 0]",42
3,we started with apps going the chicken and waf...,food,"[[54], [266], [102], [380], [194], [299], [529...","[1, 0]",43
4,the biscuits and gravy was too salty two peopl...,food,"[[299], [4], [363], [460], [439], [44], [270],...","[1, 0]",82
5,the garlic fries were a great starter (and a h...,food,"[[299], [202], [381], [6], [383], [222], [461]...","[1, 0]",24
6,our meal was excellent i had the pasta ai form...,food,"[[224], [66], [439], [215], [318], [124], [299...","[1, 0]",50
7,what i enjoy most about palo alto is so many r...,food,"[[524], [318], [231], [419], [499], [386], [8]...","[1, 0]",43
8,the drinks came out fairly quickly a good two ...,food,"[[299], [153], [221], [468], [122], [311], [38...","[1, 0]",49
9,despite the not so good burger the service was...,food,"[[362], [299], [375], [187], [193], [149], [29...","[1, 0]",82


실습을 위해 아래의 값들이 사용됩니다.  
enc_paragraph: input   
enc_category: target  
seq_length: second parameter of Dynamic RNN

In [22]:
enc_paragraph = np.array(df.enc_paragraph.tolist())
enc_category = np.array(df.enc_category.tolist())
seq_length = np.array(df.seq_length.tolist())

In [23]:
train_X = enc_paragraph

In [24]:
train_Y = enc_category

입력값의 형태를 확인해봅니다.

In [26]:
train_X.shape

(20, 91, 1)

총 20개의 학습 데이터가 91개의 인덱스로 구성되어 있음을 확인할 수 있습니다.

아래 코드를 실행하여, 총 20개의 실제값(target)이 2d 벡터 형태로 존재하는 것을 확인할 수 있습니다.  
[1, 0] : food  
[0, 1] : sports

In [28]:
train_Y.shape

(20, 2)

# 텐서플로우로 지문을 읽고 주제를 분류하는 모델 구현하기

아래 다이어그램 그대로 텐서플로우 모델을 구현해보도록 하겠습니다. 모델은 크게 두 단계로 나눠서 설명할 수 있습니다.  
1. 문맥 벡터(contextualized vector) 생성 단계  
   1) 단어를 인덱스로 변환합니다. (인덱스는 단어에 해당되는 숫자일뿐, 단어간의 유사도는 가지고 있지않습니다.)  
   2) 인덱스를 임베딩으로 변환합니다. 임베딩은 학습 과정을 통해 단어 유사도를 포함하게되어 문맥벡터 생성에 도움을 줍니다.  
   3) lstm에 임베딩된 시퀀스를 입력하여, 최종 상태값을 출력합니다. 이 최종 상태값이 문맥 벡터입니다.
2. 주제(food, sportx) 분류 단계  
   1) 문맥 벡터를 dense layer에 입력합니다.  
   2) dense layer의 출력값을 노드가 2개인 dense layer에 입력합니다.  
   3) 노드가 2개인 dense layer의 출력값을 소프트맥스에 입력시켜서, food, sports에 대한 예측값(prediction)을 구합니다.  

예측값과 실제값(target)의 크로스 엔트로피를 줄여나가는 과정을 통해 모델은 학습됩니다. 실습은 Adam Optimizer를 사용하여 학습하도록 하겠습니다.

In [21]:
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/lstm_model_overview.png", width=500, height=250)

In [30]:
tf.reset_default_graph()
tf.set_random_seed(777)

# hyperparameters
learning_rate = 0.001
n_epochs = 300

# define input
X = tf.placeholder(tf.float32, [None,max_word_cnt,1])
# define output
y = tf.placeholder(tf.int32, [None, 2])

# word embedding layer
embedding = tf.layers.dense(X, 5)
# LSTM cell
cell = tf.nn.rnn_cell.LSTMCell(num_units=64)
# get output and state (we only use state in this example)
output, state = tf.nn.dynamic_rnn(cell, embedding, dtype=tf.float32, sequence_length=seq_length)
# state is contextual vector which is input to MLP classifier (dense layer) below
dense_layer = tf.layers.dense(state.h, 32)
# logit has two nodes for cross entropy with one hot encoding target
logits = tf.layers.dense(dense_layer, 2)
# loss function
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=logits)
loss = tf.reduce_mean(cross_entropy)
# Adam Optimizor
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# 모델 요악
모델에 대한 간략한 요약은 다음과 같습니다.  
[문맥 벡터 생성]  
입력값은 단어들의 인덱스이며, 그 길이는 항상 91입니다.  
임베딩 레이어는 인덱스를 받아, 5차원 벡터의 임베딩을 출력합니다.  
LSTM 셀은 64차원 벡터의 상태값을 출력합니다.  

[문맥 벡터를 사용하여 지문의 주제 분류하기]  
주제 분류는 두개의 dense layer를 사용합니다.  
첫번째 dense layer는 32개의 노드를 가지고 있습니다.  
두번째 dense layer는 2개의 노드를 가지고 있으며, 이 2개의 노드는 소프트맥스의 입력값으로 들어갑니다.  
소프트맥스는 각 분류값에 해당할 확률을 출력합니다.

In [31]:
print(X)
print(embedding)
print(state)
print(dense_layer)
print(logits)

Tensor("Placeholder:0", shape=(?, 91, 1), dtype=float32)
Tensor("dense/BiasAdd:0", shape=(?, 91, 5), dtype=float32)
LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 64) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_4:0' shape=(?, 64) dtype=float32>)
Tensor("dense_1/BiasAdd:0", shape=(?, 32), dtype=float32)
Tensor("dense_2/BiasAdd:0", shape=(?, 2), dtype=float32)


# 학습하기

In [32]:
# initialize the variables
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:
    # Run the initializer
    sess.run(init)
    for epoch in range(1, n_epochs+1):
        sess.run(optimizer, feed_dict={X: train_X, y: train_Y})
        train_loss = sess.run(loss, feed_dict={X: train_X, y: train_Y})

        if epoch == 1 or epoch % 50 == 0:
            preds = tf.nn.softmax(logits)  # Apply softmax to logits
            correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
            # Calculate accuracy
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
            cur_acc = accuracy.eval({X: train_X, y: train_Y})
            print("epoch: "+ str(epoch)+", loss: "+str(train_loss)+", accuracy: "+str(cur_acc))

epoch: 1, loss: 0.674189, accuracy: 0.55
epoch: 50, loss: 0.436392, accuracy: 0.8
epoch: 100, loss: 0.23460719, accuracy: 0.9
epoch: 150, loss: 0.03331802, accuracy: 1.0
epoch: 200, loss: 0.0071475618, accuracy: 1.0
epoch: 250, loss: 0.0022903415, accuracy: 1.0
epoch: 300, loss: 0.0011263994, accuracy: 1.0


모든 지문들이 충분히 학습되어, 학습 정확도가 100%인 것을 확인할 수 있습니다.

# 참고자료
arXiv:1708.00107 [cs.CL]  [https://arxiv.org/abs/1708.00107]  
Learned in Translation: Contextualized Word Vectors  
Bryan McCann, James Bradbury, Caiming Xiong, Richard Socher